# The data present in the process is from Kaggle 
Link : https://www.kaggle.com/kamilpytlak/personal-key-indicators-of-heart-disease

In [16]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
database_ = pd.read_csv('heart_2020_cleaned.csv')

In [17]:
database_.describe()

,BMI,PhysicalHealth,MentalHealth,SleepTime
count,319795.000000,319795.00000,319795.000000,319795.000000
mean,28.325399,3.37171,3.898366,7.097075
std,6.356100,7.95085,7.955235,1.436007
min,12.020000,0.00000,0.000000,1.000000
25%,24.030000,0.00000,0.000000,6.000000
50%,27.340000,0.00000,0.000000,7.000000
75%,31.420000,2.00000,3.000000,8.000000
max,94.850000,30.00000,30.000000,24.000000


In [5]:
database_.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


# Restricting the sample population to 25-29 years

In [6]:
database_r = database_.loc[database_['AgeCategory'] == '25-29', :].copy()

Removing column of AgeCategory & race that I will not use for the ML model

In [7]:
del database_r['Race']
del database_r['AgeCategory']

In [8]:
database_r.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
336,No,21.40,Yes,No,No,15.0,30.0,Yes,Female,No,No,Good,6.0,Yes,No,No
436,No,20.92,Yes,Yes,No,0.0,0.0,No,Male,No,Yes,Very good,7.0,No,No,No
633,No,24.28,Yes,No,No,0.0,9.0,No,Female,No,Yes,Good,7.0,Yes,No,No
1104,No,40.35,No,No,No,3.0,5.0,No,Male,No,Yes,Good,4.0,No,No,No
1107,No,25.24,Yes,No,No,0.0,0.0,No,Male,No,Yes,Very good,7.0,No,No,No


Checking for incorrect data

In [9]:
print(str(database_r['HeartDisease'].value_counts(dropna = False)) + '\n')
print(str(database_r['Smoking'].value_counts(dropna = False)) + '\n')
print(str(database_r['AlcoholDrinking'].value_counts(dropna = False)) + '\n')
print(str(database_r['Stroke'].value_counts(dropna = False)) + '\n')
print(str(database_r['Sex'].value_counts(dropna = False)) + '\n')
print(str(database_r['DiffWalking'].value_counts(dropna = False)) + '\n')
print(str(database_r['Diabetic'].value_counts(dropna = False)) + '\n')
print(str(database_r['PhysicalActivity'].value_counts(dropna = False)) + '\n')
print(str(database_r['Asthma'].value_counts(dropna = False)) + '\n')
print(str(database_r['KidneyDisease'].value_counts(dropna = False)) + '\n')
print(str(database_r['SkinCancer'].value_counts(dropna = False)) + '\n')

No     16822
Yes      133
Name: HeartDisease, dtype: int64

No     11801
Yes     5154
Name: Smoking, dtype: int64

No     15384
Yes     1571
Name: AlcoholDrinking, dtype: int64

No     16865
Yes       90
Name: Stroke, dtype: int64

Male      8999
Female    7956
Name: Sex, dtype: int64

No     16591
Yes      364
Name: DiffWalking, dtype: int64

No                         16385
Yes                          251
No, borderline diabetes      175
Yes (during pregnancy)       144
Name: Diabetic, dtype: int64

Yes    14338
No      2617
Name: PhysicalActivity, dtype: int64

No     14087
Yes     2868
Name: Asthma, dtype: int64

No     16840
Yes      115
Name: KidneyDisease, dtype: int64

No     16872
Yes       83
Name: SkinCancer, dtype: int64



In [10]:
print(database_r['BMI'].describe())
print(database_r['PhysicalHealth'].describe())
print(database_r['MentalHealth'].describe())
print(database_r['SleepTime'].describe())

count    16955.000000
mean        27.688610
std          6.647318
min         12.850000
25%         23.030000
50%         26.390000
75%         30.900000
max         85.910000
Name: BMI, dtype: float64
count    16955.000000
mean         1.802241
std          5.211307
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         30.000000
Name: PhysicalHealth, dtype: float64
count    16955.000000
mean         5.729106
std          8.693158
min          0.000000
25%          0.000000
50%          1.000000
75%          7.000000
max         30.000000
Name: MentalHealth, dtype: float64
count    16955.000000
mean         6.986435
std          1.359409
min          1.000000
25%          6.000000
50%          7.000000
75%          8.000000
max         24.000000
Name: SleepTime, dtype: float64


In [13]:
database_r['GenHealth'].value_counts()

Very good    6484
Excellent    4940
Good         4354
Fair         1011
Poor          166
Name: GenHealth, dtype: int64

The Data Is already with some treatment level

# Transforming categorical ordinal and binary data into numeric
(HeartDisease, Smoking, AlcoholDrinking, Stroke, DiffWalking, Sex, Diabetic, PhysicalActivity, Asthma, KidneyDisease, SkinCancer)

# Binary cases

In [14]:
#Yes ou No cases
binary_yn_mapper = {"Yes": 1,"No": 0}

database_r['HeartDisease'] = database_r["HeartDisease"].replace(binary_yn_mapper)
database_r['Smoking'] = database_r["Smoking"].replace(binary_yn_mapper)
database_r['AlcoholDrinking'] = database_r["AlcoholDrinking"].replace(binary_yn_mapper)
database_r['Stroke'] = database_r["Stroke"].replace(binary_yn_mapper)
database_r['DiffWalking'] = database_r["DiffWalking"].replace(binary_yn_mapper)
database_r['PhysicalActivity'] = database_r["PhysicalActivity"].replace(binary_yn_mapper)
database_r['Asthma'] = database_r["Asthma"].replace(binary_yn_mapper)
database_r['KidneyDisease'] = database_r["KidneyDisease"].replace(binary_yn_mapper)
database_r['SkinCancer'] = database_r["SkinCancer"].replace(binary_yn_mapper)

database_r.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
336,0,21.40,1,0,0,15.0,30.0,1,Female,No,0,Good,6.0,1,0,0
436,0,20.92,1,1,0,0.0,0.0,0,Male,No,1,Very good,7.0,0,0,0
633,0,24.28,1,0,0,0.0,9.0,0,Female,No,1,Good,7.0,1,0,0
1104,0,40.35,0,0,0,3.0,5.0,0,Male,No,1,Good,4.0,0,0,0
1107,0,25.24,1,0,0,0.0,0.0,0,Male,No,1,Very good,7.0,0,0,0


In [15]:
#Non Yes or No case

binary_yn_mapper = {"Male": 0,"Female": 1}

database_r['Sex'] = database_r["Sex"].replace(binary_yn_mapper)

database_r.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
336,0,21.40,1,0,0,15.0,30.0,1,1,No,0,Good,6.0,1,0,0
436,0,20.92,1,1,0,0.0,0.0,0,0,No,1,Very good,7.0,0,0,0
633,0,24.28,1,0,0,0.0,9.0,0,1,No,1,Good,7.0,1,0,0
1104,0,40.35,0,0,0,3.0,5.0,0,0,No,1,Good,4.0,0,0,0
1107,0,25.24,1,0,0,0.0,0.0,0,0,No,1,Very good,7.0,0,0,0


# Categorical Ordinal Cases

In [19]:
genHealth_mapper = {"Poor": 0,"Fair": 1,"Good": 2,"Very good": 3,"Excellent": 4}
database_r['GenHealth'] = database_r["GenHealth"].replace(genHealth_mapper)

database_r.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
336,0,21.40,1,0,0,15.0,30.0,1,1,No,0,2,6.0,1,0,0
436,0,20.92,1,1,0,0.0,0.0,0,0,No,1,3,7.0,0,0,0
633,0,24.28,1,0,0,0.0,9.0,0,1,No,1,2,7.0,1,0,0
1104,0,40.35,0,0,0,3.0,5.0,0,0,No,1,2,4.0,0,0,0
1107,0,25.24,1,0,0,0.0,0.0,0,0,No,1,3,7.0,0,0,0


In [20]:
diabetic_mapper = {"No": 0,"No, borderline diabetes": 1,"Yes (during pregnancy)": 2,"Yes": 3}
database_r['Diabetic'] = database_r["Diabetic"].replace(diabetic_mapper)

database_r.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
336,0,21.40,1,0,0,15.0,30.0,1,1,0,0,2,6.0,1,0,0
436,0,20.92,1,1,0,0.0,0.0,0,0,0,1,3,7.0,0,0,0
633,0,24.28,1,0,0,0.0,9.0,0,1,0,1,2,7.0,1,0,0
1104,0,40.35,0,0,0,3.0,5.0,0,0,0,1,2,4.0,0,0,0
1107,0,25.24,1,0,0,0.0,0.0,0,0,0,1,3,7.0,0,0,0


Finished the data pre-process

# Separating Training and Testing data

In [23]:
train, test_and_validate = train_test_split(database_r, test_size=0.2, stratify = database_r['HeartDisease'])

Now Separating Training & validating data

In [26]:
test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['HeartDisease'])

# Training Data